In [77]:
import sys, os
sys.path.append("../../")

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.utils as vutils

from seisml.networks import Dataset, DCGAN

In [37]:
class Params:
    def __init__(self):
        self.data_dir = os.path.abspath('../../data/triggered_earthquakes/prepared/')
        self.label = 'positive'
        self.length = 8000
        self.mode = 'train'
        
        self.hidden_dim = 16
        self.output_dim = 32
        
        # Number of workers for dataloader
        self.workers = 2

        # Batch size during training
        self.batch_size = 128

        # Spatial size of training images. All images will be resized to this
        #   size using a transformer.
#         self.image_size = 64

        # Number of channels in the training images. For color images this is 3
        self.nc = 1

        # Size of z latent vector (i.e. size of generator input)
        self.nz = 100

        # Size of feature maps in generator
        self.ngf = 32

        # Size of feature maps in discriminator
        self.ndf = 32

        # Number of training epochs
        self.num_epochs = 5

        # Learning rate for optimizers
        self.lr = 0.0002

        # Beta1 hyperparam for Adam optimizers
        self.beta1 = 0.5

        # Number of GPUs available. Use 0 for CPU mode.
        self.ngpu = 1
        
params = Params()

In [38]:
dataset = Dataset(os.path.abspath('../../data/triggered_earthquakes/prepared/'), 'positive', 8000, 'train')

In [39]:
dataloader = DataLoader(
    dataset, 
    batch_size=params.batch_size,
    shuffle=True, 
    num_workers=params.workers
)

In [40]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and params.ngpu > 0) else "cpu")

In [41]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [65]:
class Generator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Fully-connected layer
        fc = torch.nn.Linear(params.output_dim, params.hidden_dim, bias=True)
        # initializer
        torch.nn.init.normal(fc.weight)
        torch.nn.init.constant(fc.bias, 0.0)

        # Hidden layer
        self.hidden_layer = torch.nn.Sequential(
            fc,
            torch.nn.ReLU()
        )

        # Output layer
        self.output_layer = torch.nn.Linear(params.hidden_dim, params.length, bias=True)
        # initializer
        torch.nn.init.normal(self.output_layer.weight)
        torch.nn.init.constant(self.output_layer.bias, 0.0)

    def forward(self, x):
        h = self.hidden_layer(x.float())
        out = self.output_layer(h)
        return out

In [66]:
# Create the generator
netG = Generator(params).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (params.ngpu > 1):
    netG = nn.DataParallel(netG, list(range(params.ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

Generator(
  (hidden_layer): Sequential(
    (0): Linear(in_features=32, out_features=16, bias=True)
    (1): ReLU()
  )
  (output_layer): Linear(in_features=16, out_features=8000, bias=True)
)


/home/b/anaconda3/envs/seismic_classifier/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  
/home/b/anaconda3/envs/seismic_classifier/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  if __name__ == '__main__':
/home/b/anaconda3/envs/seismic_classifier/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
/home/b/anaconda3/envs/seismic_classifier/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


In [67]:
class Discriminator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Fully-connected layer
        fc1 = torch.nn.Linear(params.length, params.hidden_dim, bias=True)
        # initializer
        torch.nn.init.normal(fc1.weight)
        torch.nn.init.constant(fc1.bias, 0.0)

        # Hidden layer
        self.hidden_layer = torch.nn.Sequential(
            fc1,
            torch.nn.ReLU()
        )

        # Fully-connected layer
        fc2 = torch.nn.Linear(params.hidden_dim, params.output_dim, bias=True)
        # initializer
        torch.nn.init.normal(fc2.weight)
        torch.nn.init.constant(fc2.bias, 0.0)

        # Output layer
        self.output_layer = torch.nn.Sequential(
            fc2,
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        h = self.hidden_layer(x.float())
        out = self.output_layer(h)
        return out

In [68]:
# Create the Discriminator
netD = Discriminator(params).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (params.ngpu > 1):
    netD = nn.DataParallel(netD, list(range(params.ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

Discriminator(
  (hidden_layer): Sequential(
    (0): Linear(in_features=8000, out_features=16, bias=True)
    (1): ReLU()
  )
  (output_layer): Sequential(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): Sigmoid()
  )
)


/home/b/anaconda3/envs/seismic_classifier/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  
/home/b/anaconda3/envs/seismic_classifier/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  if __name__ == '__main__':
/home/b/anaconda3/envs/seismic_classifier/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
/home/b/anaconda3/envs/seismic_classifier/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


In [75]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(params.output_dim, device=device)

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=params.lr, betas=(params.beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=params.lr, betas=(params.beta1, 0.999))

In [79]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(params.num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data['data'].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,params.output_dim), real_label, device=device).view(-1)
        # Forward pass real batch through D
        
        output = netD(real_cpu).view(-1)
        
        print(output.shape)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, params.output_dim, device=device)
        print(noise.shape)
        # Generate fake image batch with G
        fake = netG(noise)
        
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, params.num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
#         if (iters % 500 == 0) or ((epoch == params.num_epochs-1) and (i == len(dataloader)-1)):
#             with torch.no_grad():
#                 fake = netG(fixed_noise).detach().cpu()
#             img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

Starting Training Loop...
torch.Size([4096])
torch.Size([4096])
torch.Size([128, 32])
torch.Size([128, 8000])
[0/5][0/13]	Loss_D: 101.7437	Loss_G: 53.4549	D(x): 0.4422	D(G(z)): 0.4616 / 0.4605
torch.Size([4096])
torch.Size([4096])
torch.Size([128, 32])
torch.Size([128, 8000])
torch.Size([4096])
torch.Size([4096])
torch.Size([128, 32])
torch.Size([128, 8000])
torch.Size([4096])
torch.Size([4096])
torch.Size([128, 32])
torch.Size([128, 8000])
torch.Size([4096])
torch.Size([4096])
torch.Size([128, 32])
torch.Size([128, 8000])
torch.Size([4096])
torch.Size([4096])
torch.Size([128, 32])
torch.Size([128, 8000])
torch.Size([4096])
torch.Size([4096])
torch.Size([128, 32])
torch.Size([128, 8000])
torch.Size([4096])
torch.Size([4096])
torch.Size([128, 32])
torch.Size([128, 8000])
torch.Size([4096])
torch.Size([4096])
torch.Size([128, 32])
torch.Size([128, 8000])
torch.Size([4096])
torch.Size([4096])
torch.Size([128, 32])
torch.Size([128, 8000])
torch.Size([4096])
torch.Size([4096])
torch.Size([1